In [1]:
import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator

from py_biconvex_mpc.motion_planner.biconvex import BiConvexMP
from cnt_plan_utils import SoloCntGen


In [2]:
robot = Solo12Config.buildRobotWrapper()
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

n_eff = 4
m = pin.computeTotalMass(robot.model)
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [120]:
# initial and ter state
X_init = np.zeros(9)
X_init[0:3] = q0[0:3]
X_ter = X_init.copy()

# contact plan
st = 0.2 # step time 
sh = 0.1 # step height
sl = np.array([0.0,0.0,0]) # step length
n_steps = 4 # number of steps
T = st*(n_steps + 2)
dt = 1e-2

X_ter[0:3] += sl*(n_steps)

cnt_planner = SoloCntGen(T, dt, 1)
# print(cnt_plan)

In [121]:
cnt_planner.reset(T, dt)
cnt_plan = cnt_planner.create_trot_plan(st, sl, n_steps)
ik_solver = cnt_planner.create_ik_step_costs(cnt_plan, sh, [1e2, 1e3])

In [122]:
# weights
W_X = np.array([1e-5, 1e-5, 1e-5, 1e+2, 1e+2, 1e+2, 1e+4, 1e+4, 1e+4])
W_X_ter = np.array([1e+5, 1e+5, 1e+5, 1e+4, 1e+4, 1e+4, 1e+4, 1e+4, 1e+4])

W_F = np.array(4*[1e-1, 1e-1, 1e-1])

rho = 5e+2 # penalty on dynamic constraint violation

# constraints 
bx = 0.2
by = 0.2
bz = 0.32
fx_max = 15
fy_max = 15
fz_max = 15

# optimization
mp = BiConvexMP(m, dt, T, n_eff, rho = rho)
mp.create_contact_array(cnt_plan)
mp.create_bound_constraints(bx, by, bz, fx_max, fy_max, fz_max)

X_opt, F_opt, mom_opt = mp.optimize(X_init, X_ter, W_X, W_F, W_X_ter, 20)
mom_opt = np.round(mom_opt, 2)

iter number 0
iter number 1
iter number 2
iter number 3
iter number 4
iter number 5
iter number 6
iter number 7
iter number 8
iter number 9
iter number 10
iter number 11
iter number 12
iter number 13
iter number 14
iter number 15
iter number 16
iter number 17
iter number 18
iter number 19


In [123]:
ik_solver.create_centroidal_task(mom_opt, 0, T, "mom_track_cost", 1e+3)
xs = ik_solver.optimize(x0)

In [126]:
print(len(xs), T/dt)
for i in range(len(xs)):
    time.sleep(0.05)
    viz.display(xs[i][:robot.model.nq])

121 120.00000000000001


In [125]:
for i in range(len(xs)):
    q = xs[i][:robot.model.nq]
    v = xs[i][robot.model.nq:]
    pin.forwardKinematics(robot.model, robot.data, q, v)
    pin.updateFramePlacements(robot.model, robot.data)
    pin.centerOfMass(robot.model, robot.data, q, v)
    pin.computeCentroidalMomentum(robot.model, robot.data)
    print(np.round(robot.data.hg, 2)[0:3], mom_opt[i][0:3])

[0. 0. 0.] [-0.03 -0.03  0.19]
[-0.04 -0.03  0.12] [-0.04 -0.05  0.16]
[-0.05 -0.03  0.1 ] [-0.05 -0.05  0.14]
[-0.05 -0.03  0.09] [-0.05 -0.05  0.12]
[-0.05 -0.03  0.08] [-0.05 -0.05  0.11]
[-0.04 -0.02  0.06] [-0.04 -0.04  0.09]
[-0.03 -0.02  0.06] [-0.03 -0.04  0.08]
[-0.03 -0.02  0.04] [-0.03 -0.03  0.06]
[-0.02 -0.01  0.04] [-0.02 -0.02  0.05]
[-0.01 -0.    0.03] [-0.01 -0.01  0.04]
[-0.    0.    0.02] [-0.   -0.    0.03]
[-0.    0.    0.01] [0.   0.   0.02]
[0.01 0.01 0.01] [0.01 0.01 0.02]
[0.01 0.01 0.01] [0.01 0.01 0.02]
[0.01 0.01 0.01] [0.01 0.02 0.02]
[0.01 0.01 0.01] [0.01 0.01 0.02]
[0.01 0.01 0.01] [0.01 0.01 0.02]
[-0.    0.01  0.01] [0.   0.01 0.01]
[-0.01  0.   -0.  ] [-0.01 -0.    0.  ]
[-0.01  0.   -0.  ] [-0.01 -0.    0.  ]
[-0.01 -0.    0.01] [-0.01 -0.01  0.  ]
[-0.02 -0.   -0.  ] [-0.02 -0.01 -0.01]
[-0.02 -0.01 -0.  ] [-0.02 -0.02 -0.01]
[-0.03 -0.01 -0.01] [-0.03 -0.02 -0.02]
[-0.03 -0.02 -0.01] [-0.03 -0.03 -0.02]
[-0.03 -0.02 -0.02] [-0.03 -0.03 -0.03]
[-0.0